# Structure gemini outputs with Pydantic

In [2]:
from google import genai
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List a few asian soups recipes, a yummy description and list the ingredients",
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]}
)

print(response.text)

[
  {
    "recipe_name": "Tom Yum Goong",
    "description": "A classic Thai hot and sour soup, bursting with aromatic herbs like lemongrass, galangal, and kaffir lime leaves, and filled with succulent shrimp. Its spicy kick and tangy notes create an unforgettable flavor experience that's both invigorating and comforting.",
    "ingredients": [
      "Shrimp",
      "Lemongrass",
      "Galangal",
      "Kaffir lime leaves",
      "Mushrooms",
      "Fish sauce",
      "Lime juice",
      "Chili paste",
      "Cherry tomatoes",
      "Cilantro"
    ]
  },
  {
    "recipe_name": "Shoyu Ramen",
    "description": "A deeply savory Japanese noodle soup featuring a rich soy-sauce-based broth, springy ramen noodles, tender slices of chashu pork, a perfectly jammy ajitama egg, and crisp nori, creating a symphony of textures and umami.",
    "ingredients": [
      "Ramen noodles",
      "Pork belly (for chashu)",
      "Soy sauce",
      "Mirin",
      "Sake",
      "Chicken or pork broth",
  

In [4]:
recipes = response.parsed
len(recipes)

3

In [5]:
recipes[0].description

"A classic Thai hot and sour soup, bursting with aromatic herbs like lemongrass, galangal, and kaffir lime leaves, and filled with succulent shrimp. Its spicy kick and tangy notes create an unforgettable flavor experience that's both invigorating and comforting."

In [6]:
recipes[0].recipe_name

'Tom Yum Goong'

In [7]:
recipes[0].ingredients

['Shrimp',
 'Lemongrass',
 'Galangal',
 'Kaffir lime leaves',
 'Mushrooms',
 'Fish sauce',
 'Lime juice',
 'Chili paste',
 'Cherry tomatoes',
 'Cilantro']

## Simulate house prices

In [8]:
from typing import Literal

class Home(BaseModel):
    price: int 
    monthly_fee: int
    living_area: float 
    number_rooms: int 
    type: Literal["apartment", "house"]
    address: str 



response = client.models.generate_content(
    model = "gemini-2.5-pro",
    contents = """List 50 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK. Data should be simulation of swedish housing""",
    config = {"response_mime_type": "application/json", "response_schema": list[Home]}
)


homes = response.parsed
homes 

[Home(price=5250000, monthly_fee=3150, living_area=58.5, number_rooms=2, type='apartment', address='Sveavägen 45, 111 34 Stockholm'),
 Home(price=3100000, monthly_fee=2800, living_area=45.0, number_rooms=2, type='apartment', address='Linnégatan 22, 413 04 Göteborg'),
 Home(price=2500000, monthly_fee=3500, living_area=65.0, number_rooms=3, type='apartment', address='Davidshallsgatan 10, 211 45 Malmö'),
 Home(price=2800000, monthly_fee=2650, living_area=50.5, number_rooms=2, type='apartment', address='Svartbäcksgatan 30, 753 32 Uppsala'),
 Home(price=8500000, monthly_fee=4500, living_area=160.0, number_rooms=5, type='house', address='Björkvägen 8, 181 35 Lidingö'),
 Home(price=1950000, monthly_fee=1900, living_area=32.0, number_rooms=1, type='apartment', address='Storgatan 5, 903 26 Umeå'),
 Home(price=6200000, monthly_fee=3800, living_area=155.5, number_rooms=6, type='house', address='Ekollonvägen 14, 436 39 Askim'),
 Home(price=3400000, monthly_fee=3950, living_area=82.0, number_rooms=

In [10]:
homes[0].price, homes[0].address

(5250000, 'Sveavägen 45, 111 34 Stockholm')

In [13]:
homes[0].__dict__

{'price': 5250000,
 'monthly_fee': 3150,
 'living_area': 58.5,
 'number_rooms': 2,
 'type': 'apartment',
 'address': 'Sveavägen 45, 111 34 Stockholm'}

In [15]:
import pandas as pd

df = pd.DataFrame(home.__dict__ for home in homes)
df.head()

,price,monthly_fee,living_area,number_rooms,type,address
0,5250000,3150,58.5,2,apartment,"Sveavägen 45, 111 34 Stockholm"
1,3100000,2800,45.0,2,apartment,"Linnégatan 22, 413 04 Göteborg"
2,2500000,3500,65.0,3,apartment,"Davidshallsgatan 10, 211 45 Malmö"
3,2800000,2650,50.5,2,apartment,"Svartbäcksgatan 30, 753 32 Uppsala"
4,8500000,4500,160.0,5,house,"Björkvägen 8, 181 35 Lidingö"


In [20]:
cheap_houses = df.query("price < 5_000_000 and type == 'house'")
cheap_houses

,price,monthly_fee,living_area,number_rooms,type,address
9,4500000,2900,130.0,5,house,"Violvägen 3, 224 56 Lund"
18,3800000,2800,125.0,4,house,"Solrosvägen 11, 554 57 Jönköping"
20,3600000,3300,110.0,5,house,"Bergvägen 22, 981 32 Kiruna"
23,4900000,3100,145.0,6,house,"Parkvägen 7, 352 35 Växjö"
26,3200000,2500,115.0,4,house,"Skogsvägen 1, 974 31 Luleå"
34,3900000,2700,120.0,4,house,"Granbacksvägen 10, 191 43 Sollentuna"
43,3500000,2600,135.0,5,house,"Björkbacksvägen 4, 903 39 Umeå"
48,4200000,3000,160.0,6,house,"Stenhuggarvägen 7, 621 40 Visby"


In [21]:
cheap_houses.to_csv("cheap_houses.csv", index=False)